In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [25]:
# Set the path to the folders containing the images
# train_dir = './training_data/'
train_dir = "F:\\big-data\\uni\\minor-a-systems\\cucumber-dataset-compressed\\normal-light"

# Define the image size
img_height = 224
img_width = 224

# Define the batch size
batch_size = 32

# Define the number of classes
num_classes = 2


In [26]:
# Create a training dataset from the images in the training folder
(train_ds, validation_ds) = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

# Print the number of batches in the training and validation datasets
print("Number of batches in training dataset: %d" %
      tf.data.experimental.cardinality(train_ds))
print("Number of batches in validation dataset: %d" %
      tf.data.experimental.cardinality(validation_ds))


Found 2043 files belonging to 2 classes.
Using 1635 files for training.
Using 408 files for validation.
Number of batches in training dataset: 52
Number of batches in validation dataset: 13


In [27]:
train_ds
classes = train_ds.class_names

In [29]:
# Create the model
model = keras.Sequential([
    keras.layers.experimental.preprocessing.Rescaling(
        1./255, input_shape=(img_height, img_width, 3)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(128, 3, activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

# Fit the model to the training dataset
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    batch_size=batch_size,
    epochs=10
)


Epoch 1/10
52/52 [==============================] - 19s 309ms/step - loss: 0.6782 - accuracy: 0.6336 - val_loss: 0.4649 - val_accuracy: 0.7868
Epoch 2/10
52/52 [==============================] - 18s 304ms/step - loss: 0.4044 - accuracy: 0.8232 - val_loss: 0.3987 - val_accuracy: 0.8137
Epoch 3/10
52/52 [==============================] - 19s 327ms/step - loss: 0.3438 - accuracy: 0.8422 - val_loss: 0.4231 - val_accuracy: 0.8064
Epoch 4/10
52/52 [==============================] - 20s 331ms/step - loss: 0.3173 - accuracy: 0.8618 - val_loss: 0.4430 - val_accuracy: 0.8039
Epoch 5/10
52/52 [==============================] - 19s 310ms/step - loss: 0.2430 - accuracy: 0.8942 - val_loss: 0.2668 - val_accuracy: 0.8799
Epoch 6/10
52/52 [==============================] - 18s 305ms/step - loss: 0.1891 - accuracy: 0.9254 - val_loss: 0.3608 - val_accuracy: 0.8480
Epoch 7/10
52/52 [==============================] - 18s 297ms/step - loss: 0.1865 - accuracy: 0.9346 - val_loss: 0.2683 - val_accuracy: 0.8946

In [30]:
def prediction_to_class(p, classes):
    for (i, x) in enumerate(classes):
        if p < (i+1) * (1 / len(classes)):
            return x;
    return classes[-1]

In [31]:
test_img_path = './training_data/healthy/1.bmp'

# Load a new image
new_image = tf.keras.preprocessing.image.load_img(
    test_img_path, target_size=(img_height, img_width))

# Preprocess the image
new_image = tf.keras.preprocessing.image.img_to_array(new_image)
new_image = tf.expand_dims(new_image, axis=0)

# Make a prediction with the model
prediction = model.predict(new_image)
prediction = prediction_to_class(prediction, classes)
prediction

1/1 [==============================] - 0s 143ms/step


'mini-healthy'